# 3. Exploratory Data Analysis (EDA)

## 3.1 Overview of Dataset

We will look at the distributions between variables relevant to our prediction task:

Goal: Predict played_hours using experience level, subscription status, gender, and age.

## 3.2 Basic Data Checks

In [35]:
import pandas as pd

df = pd.read_csv("players.csv")

df = df.drop(columns=['hashedEmail', 'individualId', 'organizationName', 'name'])

df['subscribe'] = df['subscribe'].map({
    True: 'Subscribed',
    False: 'Not Subscribed'
})

df['gender'] = df['gender'].map({
    'Male': 'Male',
    'Female': 'Female',
    'Prefer not to say': 'Other',
    'Agender': 'Other',
    'Non-binary': 'Other',
    'Two-Spirited': 'Other',
    'Other': 'Other'
})

In [36]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   experience    196 non-null    object 
 1   subscribe     196 non-null    object 
 2   played_hours  196 non-null    float64
 3   gender        196 non-null    object 
 4   age           196 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 7.8+ KB


,experience,subscribe,played_hours,gender,age
count,196,196,196.000000,196,196.000000
unique,5,2,NaN,3,NaN
top,Amateur,Subscribed,NaN,Male,NaN
freq,63,144,NaN,124,NaN
mean,NaN,NaN,5.845918,NaN,21.280612
std,NaN,NaN,28.357343,NaN,9.706346
min,NaN,NaN,0.000000,NaN,8.000000
25%,NaN,NaN,0.000000,NaN,17.000000
50%,NaN,NaN,0.100000,NaN,19.000000
75%,NaN,NaN,0.600000,NaN,22.000000


## 3.3 Distribution of Played Hours

In [38]:
import altair as alt

fig1 = alt.Chart(df).mark_bar().encode(
    x=alt.X('played_hours', bin=alt.Bin(maxbins=40),
            title='Total Hours Played'),
    y=alt.Y('count()', title='Number of Players')
).properties(
    width=500,
    height=300,
    title='Figure 1. Distribution of Played Hours'
)

fig1

alt.Chart(...)

#### Explanation:
- Played hours are heavily right-skewed.
- Most players have extremely low played hours (often close to 0).
- Minimal number of players have very high playtime (> 100 hours).
- This suggests that the outliers will greatly influence regression performance.

## 3.4 Played Hours by Experience Level

In [40]:
fig2 = alt.Chart(df).mark_boxplot().encode(
    x=alt.X('experience', sort=['Beginner','Amateur','Regular','Veteran','Pro']),
    y='played_hours'
).properties(
    width=500,
    title='Figure 2. Played Hours by Experience Level'
)

fig2

alt.Chart(...)

#### Explanation: 
- Amateur and Regular players possess the largest outliers.
- Interestingly enough, Pro and Veteran players do not always play the most.
- Regular players occasionally have the highest recorded hours.
- This indicates experience level is not a simple linear predictor.

## 3.5 Played Hours by Subscription Status

In [41]:
fig3 = alt.Chart(df).mark_boxplot().encode(
    x=alt.X('subscribe', title='Subscription Status'),
    y='played_hours',
    color='subscribe'
).properties(
    width=400,
    title='Figure 3. Played Hours vs Subscription Status'
)

fig3

alt.Chart(...)

#### Explanation: 
- Subscribed users tend to have higher median played hours.
- A few of the heaviest users appear in the subscribed group.
- This suggests subscription status may help predict playtime.

## 3.6 Played Hours by Gender

In [42]:
fig4 = alt.Chart(df).mark_boxplot().encode(
    x='gender',
    y='played_hours',
    color='gender'
).properties(
    width=400,
    title='Figure 4. Played Hours by Gender'
)

fig4

alt.Chart(...)

#### Explanation:
- Gender does not show a strong relationship with played hours.
- Variation for each group is large.
- Some categories, such as "Other", have small sample sizes.

## 3.7 Relationship Between Age and Played Hours

In [43]:
fig5 = alt.Chart(df).mark_point(opacity=0.5).encode(
    x='age',
    y='played_hours',
    color='experience'
).properties(
    width=500,
    height=300,
    title='Figure 5. Age vs Played Hours'
)

fig5

alt.Chart(...)

#### Explanation: 
- No strong linear relationship between age and playtime.
- Heaviest players tend to be younger and often with a “Amateur” or “Regular” experience level.
- Age can contribute weak predictive signal when combined with other features.

# 3.8 Numerical Correlation

In [44]:
df[['age','played_hours']].corr()

,age,played_hours
age,1.000000,-0.050676
played_hours,-0.050676,1.000000


#### Explanation
Correlation between age and played_hours is very weak, confirming the scatterplot's patterns.

# 3.9 EDA Summary

This analysis shows that `played_hours` is highly right-skewed where many players log very 
little time and a minority contribute to over 100 hours of playtime. Experience level shows some 
patterns that reveal, despite Pro and Veteran players being more experienced, Amateur and 
Regular players often record the highest total hours. This phenomenon suggests that experience 
level does not increase playtime in a linear way. Subscription status shows a strong relationship, 
where subscribed players tend to play more on average and include several of the highest outliers, 
making this a potentially strong predictor.

Gender does not show a strong correlation with playtime, although sample sizes are variable.  
Age is also weakly correlated with playtime, though the highest-hour users tend to 
be younger and from the Amateur or Regular categories. Overall, the EDA suggests that 
experience and subscription status will be the most important predictors, while age and 
gender may offer limited additional signal.